# "[Spark] PySpark Read Write"
> pyspark에 데이터 불러오고 내보내기

- toc: true 
- badges: true
- comments: true
- categories: [Spark]
- tags: [spark, pyspark, read, write]

In [1]:
import os
MINIO_ACCESS_KEY = os.environ['MINIO_ACCESS_KEY']
MINIO_SECRET_KEY = os.environ['MINIO_SECRET_KEY']

spark.sparkContext._jsc.hadoopConfiguration()\
    .set("fs.s3a.access.key", MINIO_ACCESS_KEY)
spark.sparkContext._jsc.hadoopConfiguration()\
    .set("fs.s3a.secret.key", MINIO_SECRET_KEY)
spark.sparkContext._jsc.hadoopConfiguration()\
    .set("fs.s3a.endpoint", "http://lab101:10170")
spark.sparkContext._jsc.hadoopConfiguration()\
    .set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc.hadoopConfiguration()\
    .set("fs.s3a.connection.ssl.enabled", "false")
spark.sparkContext._jsc.hadoopConfiguration()\
    .set("fs.s3a.path.style.access", "true")
spark.sparkContext._jsc.hadoopConfiguration()\
    .set("com.amazonaws.services.s3.enableV2", "true")
spark.sparkContext._jsc.hadoopConfiguration()\
    .set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

## CSV 파일

In [6]:
csvFile = spark.read.format("csv")\
    .option("header", "true")\
    .option("mode", "FAILFAST")\
    .option("inferSchema", "true")\
    .load("s3a://data/flight-data/csv/2010-summary.csv")

In [7]:
csvFile.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [8]:
csvFile.write.format("csv").mode("overwrite").option("sep", "\t")\
    .save("s3a://tmp/my-tsv-file.csv")

## JSON 파일
- 자바스크립트 객체 표기법(JavaScript Object Notation)
- 스파크에서는 JSON 파일을 사용할 때, 줄로 구분된 JSON을 기본적으로 사용
- **multiLine** 옵션을 이용해 줄로 구분된 방식과 여러 줄로 구성된 방식을 선택적으로 사용할 수 있음
- **multiLine** 옵션을 **true**로 설정하면 전체 파일을 하나의 JSON 객체로 읽을 수 있음

In [13]:
df = spark.read.format("json").option("mode", "FAILFAST")\
    .option("inferSchema", "true")\
    .load("s3a://data/flight-data/json/2010-summary.json")
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [14]:
df.write.format("json").mode("overwrite").save("s3a://tmp/my-json-file.json")

## Parquet 파일
- 읽기 연산시 JSON이나 CSV보다 효율적으로 동작

In [18]:
df = spark.read.format("parquet")\
    .load("s3a://data/flight-data/parquet/2010-summary.parquet")
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



| 읽기/쓰기 | 키 | 사용 가능한 값 | 기본값 | 설명 |
|:---:|:---:|:---:|:---:|:---:|
| 모두 | compression 또는 codec | none, uncompressed, bzip2, deflate, gzip, deflate, gzip, lz4, snappy | none | 스파크가 파일을 읽고 쓸 때 사용하는 압축 코덱 정의 |
| 읽기 | mergeSchema | true, false | spark.sql.parquet.mergeSchema 속성의 설정 값 | 동일한 테이블이나 폴더에 신규 추가된 파케이 파일에 컬럼을 점진적으로 추가할 수 있음 |

In [19]:
df.write.format("parquet").mode("overwrite").save("s3a://tmp/my-parquet-file.parquet")

## ORC 파일
- 하둡 워크로드를 위해 설계된 self-describing이며, 데이터 타입을 인식할 수 있는 컬럼 기반의 파일 포맷
- 대규모 스트리밍 읽기에 최적화
- 필요한 로우를 신속하게 찾아낼 수 있는 기능이 통합
- 파케이가 스파크에 최적화된 반면 ORC는 하이브에 최적화

In [20]:
df = spark.read.format("orc").load("s3a://data/flight-data/orc/2010-summary.orc")
df.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
|    United States|              India|   69|
|            Egypt|      United States|   24|
|Equatorial Guinea|      United States|    1|
+-----------------+-------------------+-----+
only showing top 5 rows



In [21]:
df.write.format("orc").mode("overwrite").save("s3a://tmp/my-orc-file.orc")

## 텍스트 파일

In [356]:
txtFile = spark.read.format("text").load("s3a://data/flight-data/csv/2010-summary.csv").selectExpr("split(value, ',') as rows")
txtFile.show(5)

+--------------------+
|                rows|
+--------------------+
|[DEST_COUNTRY_NAM...|
|[United States, R...|
|[United States, I...|
|[United States, I...|
|[Egypt, United St...|
+--------------------+
only showing top 5 rows



In [358]:
csvFile_list = list(txtFile.toPandas()['rows'])
csvFile_df = pd.DataFrame(csvFile_list[1:], columns = csvFile_list[:1][0] + ["_3", "_4"])
csvFile = spark.createDataFrame(csvFile_df)
csvFile.show(5)

+-----------------+-------------------+-----+----+----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|  _3|  _4|
+-----------------+-------------------+-----+----+----+
|    United States|            Romania|    1|null|null|
|    United States|            Ireland|  264|null|null|
|    United States|              India|   69|null|null|
|            Egypt|      United States|   24|null|null|
|Equatorial Guinea|      United States|    1|null|null|
+-----------------+-------------------+-----+----+----+
only showing top 5 rows



- 텍스트 파일을 쓸 때는 문자열 컬럼이 하나만 존재해야 함

In [360]:
csvFile.select("DEST_COUNTRY_NAME").write.text("s3a://tmp/simple-text-file.txt")

- 텍스트 파일에 데이터를 저장할 때 파티셔닝 작업을 수행하면 더 많은 컬럼을 저장 가능
- 모든 파일에 컬럼을 추가하는 것이 아니라 텍스트 파일이 저장되는 디렉터리에 폴더 별로 컬름을 저장함

In [361]:
csvFile.limit(10).select("DEST_COUNTRY_NAME", "count")\
    .write.partitionBy("count").text("s3a://tmp/five-csv-files2py.csv")

## ML Model

In [3]:
from pyspark.ml.feature import PCA

scaleDF = spark.read.parquet("s3a://data/simple-ml-scaling")
pca = PCA().setInputCol("features").setK(2)
fittedPCA = pca.fit(scaleDF)

In [4]:
# ML Model Save
fittedPCA.write().overwrite().save("s3a://tmp/fittedPCA")

In [5]:
# ML Model Load
from pyspark.ml.feature import PCAModel

loadedPCA = PCAModel.load("s3a://tmp/fittedPCA")
loadedPCA.transform(scaleDF).show()

+---+--------------+-----------------------------+
| id|      features|PCAModel_06b47752a026__output|
+---+--------------+-----------------------------+
|  0|[1.0,0.1,-1.0]|         [0.07137194992484...|
|  1| [2.0,1.1,1.0]|         [-1.6804946984073...|
|  0|[1.0,0.1,-1.0]|         [0.07137194992484...|
|  1| [2.0,1.1,1.0]|         [-1.6804946984073...|
|  1|[3.0,10.1,3.0]|         [-10.872398139848...|
+---+--------------+-----------------------------+



# 고급 I/O 개념

## 병렬로 데이터 쓰기
- 데이터 파티션 당 하나의 파일이 작성

In [362]:
# 폴더 안에 5개의 파일 생성
csvFile.repartition(5).write.format("csv").save("s3a://tmp/multiple.csv")

## 파티셔닝
- 파티셔닝은 어떤 데이터를 어디에 저장할 것인지 제어할 수 있는 기능
- 데이터를 읽을 때 전체 데이터셋을 스캔하지 않고 필요한 컬럼의 데이터만 읽을 수 있음

In [363]:
csvFile.limit(10).write.mode("overwrite").partitionBy("DEST_COUNTRY_NAME")\
    .save("s3a://tmp/partitioned-files.parquet")

## 버켓팅
- 각 파일에 저장된 데이터를 제어할 수 있는 또 다른 파일 조직화 기법
- 버켓팅을 사용하면 동일한 버킷 ID를 가진 데이터가 하나의 물리적 파티션에 모두 모여 있기 때문에 데이터를 읽을 때 셔플을 피할 수 있음
- 버켓팅은 스파크 관리 테이블에서만 사용 가능

```python
number_buckets = 10
column_to_bucket_by = "count"

csvFile.write.format("parquet").mode("overwrite")\
    .bucketBy(number_buckets, column_to_bucket_by).saveAsTable("bucketFiles")
```

## 파일 크기 관리
- 파일당 최대 5000개의 로우를 포함하도록 보장

```python
df.write.option("maxRecordPerFile", 5000)
```

## MySQL

```sh
$ ./bin/spark-shell --driver-class-path mysql-connector-java-8.0.17.jar --jars mysql-connector-java-8.0.17.jar
```